### Spark session configuration
This cell sets Spark session settings to enable _Verti-Parquet_ and _Optimize on Write_. More details about _Verti-Parquet_ and _Optimize on Write_ in tutorial document.

In [6]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.

spark.conf.set("sprk.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824")

StatementMeta(, 130188a5-6608-4480-97e7-d80e27ce6821, 8, Finished, Available, Finished)

### Fact - Sale

This cell reads raw data from the _Files_ section of the lakehouse, adds additional columns for different date parts and the same information is being used to create partitioned fact delta table.

In [7]:
from pyspark.sql.functions import col, year, month, quarter

table_name = 'fact_sale'

# Update the format to CSV and set options for header and schema inference
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('Files/0e7c2ef8-0e90-4aa3-8f71-551c25cc3016/WideWorldImportersDW/csv/full/fact_sale_1y_full')

df = df.withColumn('Year', year(col("InvoiceDateKey")))
df = df.withColumn('Quarter', quarter(col("InvoiceDateKey")))
df = df.withColumn('Month', month(col("InvoiceDateKey")))

df.write.mode("overwrite").format("delta").partitionBy("Year", "Quarter").save("Tables/" + table_name)


StatementMeta(, 130188a5-6608-4480-97e7-d80e27ce6821, 9, Finished, Available, Finished)

### Dimensions
This cell creates a function to read raw data from the _Files_ section of the lakehouse for the table name passed as a parameter. Next, it creates a list of dimension tables. Finally, it has a _for loop_ to loop through the list of tables and call above function with each table name as parameter to read data for that specific table and create delta table.

In [2]:
def loadFullDataFromSource(table_name):
    path = "Files/0e7c2ef8-0e90-4aa3-8f71-551c25cc3016/WideWorldImportersDW/csv/full/" + table_name
    # Read CSV with header and schema inference
    df = (
        spark.read
             .format("csv")
             .option("header", "true")
             .option("inferSchema", "true")
             .load(path)
    )
    
    # Special case for dimension_customer which seems to have a more complex duplication issue
    if table_name == 'dimension_customer':
        # List all columns and manually rename them to ensure uniqueness
        print(f"Column names for {table_name}: {df.columns}")
        
        # Select each column with explicit aliases to avoid duplicates
        columns_with_aliases = [f"`{col}` as `{col}_{i}`" for i, col in enumerate(df.columns)]
        select_expr = ", ".join(columns_with_aliases)
        df = df.selectExpr(*columns_with_aliases)
    else:
        # Standard duplicate column handling for other tables
        cols = df.columns
        new_cols = []
        seen = {}
        for name in cols:
            if name in seen:
                seen[name] += 1
                new_cols.append(f"{name}_{seen[name]}")
            else:
                seen[name] = 0
                new_cols.append(name)
        
        # Rename DataFrame columns if duplicates were found
        if new_cols != cols:
            print(f"Renaming columns for table '{table_name}':")
            print("Original:", cols)
            print("New:     ", new_cols)
            df = df.toDF(*new_cols)
    
    # Write the DataFrame to Delta format
    df.write.mode("overwrite").format("delta").save("Tables/" + table_name)

# List of dimension tables to ingest
dimension_tables = [
    'dimension_city',
    'dimension_customer',
    'dimension_date',
    'dimension_employee',
    'dimension_stock_item'
]

for table in dimension_tables:
    try:
        loadFullDataFromSource(table)
        print(f"Successfully loaded {table}")
    except Exception as e:
        print(f"Error loading {table}: {str(e)}")

StatementMeta(, e83de5de-6890-4da9-9b6e-eb734c0fdd75, 4, Finished, Available, Finished)

Successfully loaded dimension_city
Column names for dimension_customer: ['CustomerKey', 'WWICustomerID', 'Customer', 'BillToCustomer', 'Category', 'BuyingGroup', 'PrimaryContact', 'PostalCode', 'ValidFrom', 'ValidTo', 'LineageKey']
Error loading dimension_customer: [_LEGACY_ERROR_TEMP_DELTA_0007] A schema mismatch detected when writing to the Delta table (Table ID: f128e820-453c-487a-9d79-2d4335ba0d89).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- CustomerKey: long (nullable = true)
-- WWICustomerID: long (nullable = true)
-- Customer: string (nullable = true)
-- BillToCustomer: string (nullable = true)
-- Category: string (nullable = true)
-- BuyingGroup: string (nullable = true)
-- PrimaryContact: string (nullable = true)


In [3]:
# Register dimension tables
spark.sql("CREATE OR REPLACE VIEW wwilakehouse.dimension_city AS SELECT * FROM delta.`Tables/dimension_city`")
spark.sql("CREATE OR REPLACE VIEW wwilakehouse.dimension_date AS SELECT * FROM delta.`Tables/dimension_date`")
spark.sql("CREATE OR REPLACE VIEW wwilakehouse.dimension_employee AS SELECT * FROM delta.`Tables/dimension_employee`")

# Register fact table
spark.sql("CREATE OR REPLACE VIEW wwilakehouse.fact_sale AS SELECT * FROM delta.`Tables/fact_sale`")

StatementMeta(, e83de5de-6890-4da9-9b6e-eb734c0fdd75, 5, Finished, Available, Finished)

AnalysisException: [SCHEMA_NOT_FOUND] The schema `wwilakehouse` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a catalog, verify the current_schema() output, or qualify the name with the correct catalog.
To tolerate the error on drop use DROP SCHEMA IF EXISTS.

In [4]:
# First create the schema
spark.sql("CREATE SCHEMA IF NOT EXISTS wwilakehouse")

# Then register dimension tables
spark.sql("CREATE OR REPLACE VIEW wwilakehouse.dimension_city AS SELECT * FROM delta.`Tables/dimension_city`")
spark.sql("CREATE OR REPLACE VIEW wwilakehouse.dimension_date AS SELECT * FROM delta.`Tables/dimension_date`")
spark.sql("CREATE OR REPLACE VIEW wwilakehouse.dimension_employee AS SELECT * FROM delta.`Tables/dimension_employee`") 

# Register fact table
spark.sql("CREATE OR REPLACE VIEW wwilakehouse.fact_sale AS SELECT * FROM delta.`Tables/fact_sale`")

StatementMeta(, e83de5de-6890-4da9-9b6e-eb734c0fdd75, 6, Finished, Available, Finished)

Py4JJavaError: An error occurred while calling o341.sql.
: java.lang.RuntimeException: java.lang.reflect.InvocationTargetException
	at com.microsoft.azure.trident.spark.TridentCoreProxy.failCreateDbIfTrident(TridentCoreProxy.java:275)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalog.createDatabase(SessionCatalog.scala:309)
	at org.apache.spark.sql.execution.datasources.v2.V2SessionCatalog.createNamespace(V2SessionCatalog.scala:327)
	at org.apache.spark.sql.connector.catalog.DelegatingCatalogExtension.createNamespace(DelegatingCatalogExtension.java:163)
	at org.apache.spark.sql.execution.datasources.v2.CreateNamespaceExec.run(CreateNamespaceExec.scala:47)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:43)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result(V2CommandExec.scala:43)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.executeCollect(V2CommandExec.scala:49)
	at org.apache.spark.sql.

In [5]:
# Use direct table references instead
df_fact_sale = spark.read.table("delta.`Tables/fact_sale`") 
df_dimension_date = spark.read.table("delta.`Tables/dimension_date`")
df_dimension_city = spark.read.table("delta.`Tables/dimension_city`")
df_dimension_employee = spark.read.table("delta.`Tables/dimension_employee`")

StatementMeta(, e83de5de-6890-4da9-9b6e-eb734c0fdd75, 7, Finished, Available, Finished)

In [6]:
# Create temporary views
df_fact_sale.createOrReplaceTempView("fact_sale")
df_dimension_date.createOrReplaceTempView("dimension_date")
df_dimension_city.createOrReplaceTempView("dimension_city")
df_dimension_employee.createOrReplaceTempView("dimension_employee")

StatementMeta(, e83de5de-6890-4da9-9b6e-eb734c0fdd75, 8, Finished, Available, Finished)

In [7]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW sale_by_date_employee
AS
SELECT
    DD.Date, DD.CalendarMonthLabel
    , DD.Day, DD.ShortMonth Month, CalendarYear Year
    ,DE.PreferredName, DE.Employee
    ,SUM(FS.TotalExcludingTax) SumOfTotalExcludingTax
    ,SUM(FS.TaxAmount) SumOfTaxAmount
    ,SUM(FS.TotalIncludingTax) SumOfTotalIncludingTax
    ,SUM(Profit) SumOfProfit 
FROM fact_sale FS
INNER JOIN dimension_date DD ON FS.InvoiceDateKey = DD.Date
INNER JOIN dimension_employee DE ON FS.SalespersonKey = DE.EmployeeKey
GROUP BY DD.Date, DD.CalendarMonthLabel, DD.Day, DD.ShortMonth, DD.CalendarYear, DE.PreferredName, DE.Employee
ORDER BY DD.Date ASC, DE.PreferredName ASC, DE.Employee ASC

StatementMeta(, e83de5de-6890-4da9-9b6e-eb734c0fdd75, 9, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>